In [1]:
import warnings
warnings.simplefilter("ignore",DeprecationWarning)
import pandas as pd
import glob
import os
from sklearn.preprocessing import  MinMaxScaler

#Путь к исходным размеченным данным
data_path='E:\Python DA\Arc Data'

In [2]:
#Функция нормализация данных в интервале [0,1]
def normalized_data(path):
    #Обозначем шаблон файла
    file_pattern=path+'\\*.csv'
    #Из директории модулем glob добавляем все файлы, соответвующее шаблону
    all_files = glob.glob(file_pattern) 
    # Указываем путь для дирректории для сохранения и создаем ее сли ее нет
    save_path = 'E:\Python DA\\Normalized Data\\all'
    os.makedirs(save_path, exist_ok=True)

    for file in all_files:
        df=pd.read_csv(file, index_col=0) #Получаем файл
        if 'IR' in df.columns:
            df.pop('IR') #Удаляем столбец IR из данных

        numeric_columns=['Ip', 'Vp', 'Vf']
        scaler = MinMaxScaler()
        df[numeric_columns]= scaler.fit_transform(df[numeric_columns])

        output_filepath = os.path.join(save_path, f'norm_{os.path.basename(file)}') #Указываем полный путь сохранения файла
        df.to_csv(output_filepath, index=False)

    return save_path #Возвращаем путь сохранения данных

In [3]:
#Функция получения скользящего среднего в данных
def MA_data(path, window_size):
    file_pattern=path+'\\*.csv'
    all_files = glob.glob(file_pattern) 
    save_path = 'E:\Python DA\\Normalized MA data\\all'
    os.makedirs(save_path, exist_ok=True)

    for file in all_files:
        df = pd.read_csv(file)
        df = df.dropna() 
        for col in ['Ip', 'Vp', 'Vf']:
            df[col] = df[col].rolling(window=window_size).mean() #Проходим скользящим средним по данным окном размера windo_size

        output_filepath = os.path.join(save_path, f'MA_{os.path.basename(file)}')

        df.to_csv(output_filepath, index = False)


    return save_path

In [4]:
#Функция для получения данных заданного объема. На вход подается список директорий и список разделения данных на необходимок оличество
def split_data(list_of_paths, list_of_sizes):
    for path in list_of_paths: #Перебираем каждый элемент из списка дирректорий
        file_pattern=path+'\\*.csv'
        all_files = glob.glob(file_pattern) 
        for file in all_files: #Перебираем каждый файл в директории
            df=pd.read_csv(file)
            df = df.dropna() 
            df_1= df[df['Arc']==1] #Все данные, которые являются горящей дугой
            df_2= df[df['Arc']==2] #Все данные, которы являются погасшей дугой

            start_index = min(df_1.index.min(), df_2.index.min()) #Начало интревала с дугой
            end_index = max(df_1.index.max(), df_2.index.max()) #Конец интервала с дугой

            mid_index = (start_index + end_index) // 2 #Середина интервала
            for size in list_of_sizes: #Для каждого размера окна данных
                
                start_selection = max(0, mid_index - size//2) #Начало окна
                end_selection = mid_index + size//2 #Конец окна
                selected_data = df.iloc[start_selection:end_selection] 

                save_path = os.path.join(os.path.dirname(path), f'{size//1000}k')
                os.makedirs(save_path, exist_ok=True)
                selected_data.to_csv(os.path.join(save_path,f'{size//1000}k_{os.path.basename(file)}'), index = False)
                        


In [6]:
#Функция для преобразования всех файлов в бинарной классифкации
def binary_data(path,exclude_paths):
    list_of_paths = [f.path for f in os.scandir(path) if f.is_dir() and f.path not in exclude_paths] #Получаем список каталоов, находящихся в path за исключением папок, храящихся в exclude_paths
    for dirpath in list_of_paths:
        list_of_subdirs = [f.path for f in os.scandir(dirpath) if f.is_dir()] #Получаем подкаталоги из списка каталогов
        for directory in list_of_subdirs:
            file_pattern = directory+'\\*.csv'
            all_files=glob.glob(file_pattern)
            for file in all_files:
                df=pd.read_csv(file)
                df = df.dropna() 
                df['Arc'] = df['Arc'].replace(2,1) #Заменяем в столбце Arc класс 2 на класс 1
                save_path = os.path.join(os.path.dirname(dirpath), f'Binary {os.path.basename(dirpath)}\\{os.path.basename(directory)}')
                os.makedirs(save_path, exist_ok=True)
                df.to_csv(os.path.join(save_path, f'Binary_{os.path.basename(file)}'), index = False)


In [9]:
#Функция, которая совмещает файлы данных в один, кроме одного. В этом файле удаляется столбец Arc. Этот файл в дальнейший будет использоваться для проверки разметки.
def combined_data(path, exclude_paths):
    list_of_paths = [f.path for f in os.scandir(path) if f.is_dir() and f.path not in exclude_paths] #Получаем список каталоов, находящихся в path за исключением папок, храящихся в exclude_paths
    for dirpath in list_of_paths:
        list_of_subdirs = [f.path for f in os.scandir(dirpath) if f.is_dir()] #Получаем подкаталоги из списка каталогов
        for directory in list_of_subdirs:
            file_pattern = directory+'\\*.csv'
            all_files=glob.glob(file_pattern)
            for i in range(len(all_files)):
                dataframes = []
                for j, file in enumerate(all_files):
                    if j!=i:
                        df = pd.read_csv(file)
                        df = df.dropna() 
                        dataframes.append(df)

                combined_df= pd.concat(dataframes, ignore_index=True)
                
                save_path = os.path.join(os.path.dirname(directory), f'Combined {os.path.basename(directory)}')
                os.makedirs(save_path, exist_ok=True)
                combined_df.to_csv(os.path.join(save_path, f'Combined_wo_{os.path.basename(all_files[i])}'), index = False)

                df=pd.read_csv(all_files[i])
                df = df.drop(columns=['Arc'])
                save_path1 = os.path.join(os.path.dirname(directory), f'WO Arc {os.path.basename(directory)}')
                os.makedirs(save_path1, exist_ok=True)
                output_fpath=os.path.join(save_path1, f'Wo_arc_{os.path.basename(all_files[i])}') 
                df.to_csv(output_fpath, index=False)

In [10]:
split_data([normalized_data(data_path),MA_data(normalized_data(data_path),100)],[10000,25000,50000,100000])
binary_data(os.path.dirname(data_path),['Arc Data'])
combined_data(os.path.dirname(data_path),['Arc Data'])